# Transformers, ¿qué pueden hacer?
Transformers es el nombre de la libreria principal de Huggingface. (hf)

Basicamente esta enfocada en facilitar el uso de modelos preentrenados por otros usuarios, o para archivar nuestros propios modelos preentrenados para usarlos en nuestros propios proyectos o compartirlo con otros.

Casi todos los modelos que tienen son modelos Transformer, no confundir con el nombre de la libreria en si. aunque la relación es evidente.

## checkpoint
Un modelo preentrenado se le llama checkpoint. Sabemos que un modelo es un programa que puede aprender en base a entrenarlo con unos datos. 

El mismo programa modelo puede ser más o menos efectivo dependiendo de la información facilitada al entrenamiento, o el numero de epocas de entrenamiento, etc... Al final el resultado de ese entrenamiento  se guarda como unos 'numeros' que representan pesos y sesgos.  Eso es un checkpoint. 

Una primera forma de verlo es que despues de x epocas de entrenamiento, guardes los datos de pesos y sesgos aprendidos como 'copia de seguridad' de lo aprendido hasta ese momento. En ese punto podriamos parar el entrenamiento y continuar posteriormente.
Si continuamos el entrenamiento con los mismos datos y condiciones que antes, seguiriamos entrenando el modelo como si no hubieramos parado, buscando minificar la funcion de perdida del modelo.

Cambiar los datos de entrenamiento o condiciones anteriores puede tener efectos no deseados segun el modelo, sobre todo porque puede 'olvidar' datos anteriores o haber un sobreentrenamiento. 

Añadir más datos al conjunto de entrenamiento suele tener mas beneficios, pero una vez que has empezado a entrenar un modelo con unos datos, cualquier cambio en las condiciones o datos de entrenamiento puede tener resultados imprevisibles tanto buenos como malos.

### fine-tunning
La mayoria de los checkpoint admiten un reentrenamiento, al menos parcial, a eso le llaman fine-tunning.
Ese finetunning  tambien pueden ser de muchos tipos. Y rara vez consiste en continuar entrenando con mas epocas el chechpoint con los mismos datos o parecidos. 

Tampoco es trivial tomar un modelo preentrenarlo y aplicarle un nuevo entrenamiento con nuevos datos que nos interesen . Corremos el riesgo que 'desaprenda' cosas anteriores o si insistimos con datos muy concretos haya un sobre-entrenamiento (haria muy bien las cosas con datos muy concretos, pero mal con datos más reales o genericos)

Depende mucho del diseño del propio modelo, unos se deberan reentrenar de una manera conjunta y otros de forma parcial. 

Tampoco es facil tomar un modelo grande y aplicarle una simple epoca mas de entrenamiento. Necesitariamos  recursos de hardware equivalentes como entrenarlos desde 0.

### LoRA
Una forma muy usada de fine-tunning es con adaptadores LoRA, o Adaptación de Bajo Rango (Low-Rank Adaptation), Se puede ver como un añadido que le hacemos al modelo original sin cambiarlo directamente. Lo de Bajo rango tiene que ver que lo hemos entrenado con unas 'dimensiones' más pequeñas que el modelo principal, (menos datos o menos dimensiones de los pesos y sesgos, etc). 
El modelo debe estar diseñado para admitir esos adaptadores LoRA, es decir no se puede hacer con todos los modelos.



HF Transformer facilita el hacer estos fine-tunning de modelos basicos con un entrenamiento generico o hacer adaptadores especializados para modelos abiertos.

Instala Transformers, Datasets, y Evaluate.

In [1]:
#!pip install datasets evaluate transformers[sentencepiece]

In [2]:
# para poderse loguear a hf desde el notebook
from huggingface_hub import notebook_login
# quitar comentario para hace login
# notebook_login()


## pipeline
pipeline() es la forma más fácil de usar un modelo preentrenado para una tarea dada.
El pipeline() soporta muchas tareas comunes y modelos listos para usar:

La palabra pipeline, tuberia, hace alusión a que crea un flujo de informacion  que pasa por la tuberia de procesos y sale un resultado.

La tarea mas sencilla seria hacer inferencia con un modelo. Otras tareas mas complejas serian entrenar un modelo, testear un modelo, hacer un fine-tunning, etc..



In [3]:
from transformers import pipeline

# Importa pipeline() , especificando el tipo de modelo a usar:
clasificador = pipeline("sentiment-analysis", device=-1) # -1= CPU # device=0 (cuda gpu)
clasificador("me gusta la gente que  comprende HuggingFace ")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


E:\python\python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[{'label': 'POSITIVE', 'score': 0.9366181492805481}]

El pipeline descarga y almacena en caché el modelo preentrenado y tokeniza para análisis de sentimiento. 
Como no hemos expecificado un  modelo concreto el pipeline 'elige'  uno por defecto.
al ser un modelo generico con diccionario ingles, quizas no es bueno clasificando frases más complicadas en español.

In [4]:
clasificador("casi termino de comprender lo bueno que son los transformers")

[{'label': 'NEGATIVE', 'score': 0.9380154013633728}]

Podemos especificar un modelo preentrenado o checkpoint diferente al de por defecto. 
Pueden estar basados en diferentes modelos o son fine tunings distintos del mismo modelo .

En cualquier caso todos los modelos que sean del grupo sentiment-analysis se pueden usar de forma muy similar


In [5]:
from transformers import pipeline

clasificador = pipeline("sentiment-analysis", model="pysentimiento/robertuito-sentiment-analysis")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [6]:
clasificador("casi termino de comprender lo bueno que son los transformers")

[{'label': 'POS', 'score': 0.8589496612548828}]

Nota: En este 2 modelo de sentimientos en vez de ser las etiquietas de resultado POSITIVE o NEGATIVE son POS o NEG.
Simplemente ilustra que pueden haber pequeñas diferencias entre modelos del mismo tipo. Se parecerán, pero cada uno tendrá unas caracteristicas, concretas

In [7]:
clasificador(
    [" HuggingFace es lo que espero.", "odio mucho esto!"]
)

[{'label': 'POS', 'score': 0.6889551281929016},
 {'label': 'NEG', 'score': 0.8335683345794678}]

Como conclusión, un pipeline facilita y simplifica al maximo el uso de un modelo, de forma que abstrae la funcionalidad de los modelos,
ocultando los detalles más especificos.
Son un buen punto de partida para probar un modelo, y lo normal es que si quieres mejorar o hacer un fine-tunning de alguno de ellos,
tienes que estudiar y conocer más detalles.




## partes de un transformer

Un transformer tiene 2 partes separadas y pueden ser más o menos independientes. 
* tokenizador 
* modelo

El tokenizador es el resposable de convertir a numeros los datos de entrada. Los datos pueden ser palabras, imagen, sonido, series...
Hay muchas estrategias de tokenizacion, incluso si solo tenemos en cuenta palabras. Es toda una ciencia la tokenicacion de cosas.
O incluso la tokenizacion de conceptos multi-modales.

### atencion?
El modelo es la parte de red neuronal en si. que suelen ser muchas capas de red 'tradicionales' en los que los datos usan un flujo forward/back 
y intercaladas unas capas llamadas de atencion en la que hay cierto flujo en la propia capa de forma que neuronas más cercanas se influyen 
con cierto peso que le llaman atención. Esa atencion tambien se puede ver como un tercer miembro del equipo peso , sesgo y atención.

El resulado de la inferencia con el modelo, sera de nuevo una lista de numeros, y se usara de nuevo el tokenizador pero a la inversa para 
convertir esos numeros el palabras (o imagen, o sonido...). 
Es decir el tokenizador tiene la funcion de codificar y decodificar los datos que entran y salen del modelo
    
Que me perdonen los cientificos de datos las simplificaciones, omisiones y errores. 


# tokenizar

Imaginemos un texto y queremos dividirlos en palabras sueltas. Una primera forma que podemos hacer es simplemente divididir por espacios la frase completa

In [8]:
texto ="antonio alcázar likes transformer network . And transformer likes attention"
tokenized_text = texto.split()
print(tokenized_text)

['antonio', 'alcázar', 'likes', 'transformer', 'network', '.', 'And', 'transformer', 'likes', 'attention']


Podriamos hacer un diccionario con las palabras de esa frase

In [9]:
diccionario = sorted(set(tokenized_text))
vocab_size = len(diccionario)
print(vocab_size)

vocab = {token:integer for integer,token in enumerate(diccionario)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

8
('.', 0)
('And', 1)
('alcázar', 2)
('antonio', 3)
('attention', 4)
('likes', 5)
('network', 6)
('transformer', 7)


En realidad un diccionario asi es demasiado pequeño para que tenga una utilidad, 
lo normal es usar un diccionario mucho más grande y que sea más o menos estandar para poderse usar en diferentes modelos.
Por otro lado, por más grande que sea un diccionario de palabras siempre podriamos encrontrar una palabra nueva, bien por ser de otro idioma, bien por ser una palabra compuesta o un neologismo. La forma de gestionar esas palabras nuevas es lo que diferencia principalmente los algoritmos de tokenizacion.
Tampoco interesa tener diccionarios infinitamente grandes, primero por el propio rendimiento, a mas numeros y datos más lento.Y sobre todo porque hay palabras de igual raiz semantica, por ejemplo vervos, que puede interesar descomponer para usar menos tokens, o evitar tener la misma palabra en singular y plural. O si es una palabra compuesta, masculino/femenino, etc..



In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = texto
tokens = tokenizer.tokenize(sequence)

print(tokens)

['ant', '##oni', '##o', 'al', '##c', '##á', '##zar', 'likes', 'transform', '##er', 'network', '.', 'And', 'transform', '##er', 'likes', 'attention']


In [11]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

[22904, 11153, 1186, 2393, 1665, 5589, 8950, 7407, 11303, 1200, 2443, 119, 1262, 11303, 1200, 7407, 2209]


In [12]:
decoded_string = tokenizer.decode(ids)
print(decoded_string)

antonio alcázar likes transformer network. And transformer likes attention


from transformers import AutoTokenizer
from transformers import AutoModel

checkpoint = "pysentimiento/robertuito-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
raw_inputs = [
   " HuggingFace es lo que espero.", "odio mucho esto!"
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

# checkpoint = "pysentimiento/robertuito-sentiment-analysis"

model = AutoModel.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

tokenizer.save_pretrained("directorio_en_mi_computador")

In [13]:
from transformers import pipeline
texto= """
    Texto: 
Análisis de Sentimiento (sentiment-analysis): clasifica la polaridad de un texto dado. 
    
Generación de Texto (Text Generation): genera texto a partir de un input dado. 
Reconocimiento de Entidades (Name Entity Recognition o NER, en inglés): etiqueta cada palabra con la entidad que representa (persona, fecha, ubicación, etc.). Responder Preguntas (Question answering, en inglés): extrae la respuesta del contexto dado un contexto y una pregunta.

Rellenar Máscara (Fill-mask, en inglés): rellena el espacio faltante dado un texto con palabras enmascaradas. Resumir (Summarization, en inglés): genera un resumen de una secuencia larga de texto o un documento. Traducción (Translation, en inglés): traduce un texto a otro idioma.

Extracción de Características (Feature Extraction, en inglés): crea una representación tensorial del texto.

Imagen:

Clasificación de Imágenes (Image Classification, en inglés): clasifica una imagen. Segmentación de Imágenes (Image Segmentation, en inglés): clasifica cada pixel de una imagen. Detección de Objetos (Object Detection, en inglés): detecta objetos dentro de una imagen.

Audio:

Clasificación de Audios (Audio Classification, en inglés): asigna una etiqueta a un segmento de audio. Reconocimiento de Voz Automático (Automatic Speech Recognition o ASR, en inglés): transcribe datos de audio a un texto.
"""


summarizer = pipeline("summarization", device=-1)
summarizer(texto)



No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'summary_text': ' Análisis de Sentimiento (sentiment-analysis) clasifica la polaridad of un texto dado . The tool clasifies the polaridad and analyzes the texto . Texto is a texto a partir of un input dado. The tool is a tool that generates texto, rather than an algorithm .'}]


#summarizer = pipeline("summarization", model="Narrativa/bsc_roberta2roberta_shared-spanish-finetuned-mlsum-summarization")
#summarizer(texto)



In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", device=-1)
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

In [ ]:
from transformers import TFAutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModel.from_pretrained(checkpoint)


In [ ]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape)

In [ ]:
from transformers import TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(inputs)
print(outputs.logits.shape)

In [ ]:
print(outputs.logits)

In [ ]:
import tensorflow as tf

predictions = tf.math.softmax(outputs.logits, axis=-1)
print(predictions)
model.config.id2label

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", device=-1)
generator("en este curso vamos a enseñar algo de inteligencia artificial")

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2", device=-1)
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

In [ ]:
from transformers import pipeline

unmasker = pipeline("fill-mask", device=-1)
unmasker("This course will teach you all about <mask> models.", top_k=2)

In [ ]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True, device=-1)
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering", device=-1)
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", device=-1)
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
"""
)

In [ ]:
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en", device=-1)
translator("Ce cours est produit par Hugging Face.")

# Sesgos y limitaciones

In [ ]:
from transformers import pipeline

unmasker = pipeline("fill-mask", model="bert-base-uncased", device=0)
result = unmasker("This man works as a [MASK].")
print([r["token_str"] for r in result])

result = unmasker("This woman works as a [MASK].")
print([r["token_str"] for r in result])

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForCTC

processor = AutoProcessor.from_pretrained("cpierse/wav2vec2-large-xlsr-53-esperanto")
model = AutoModelForCTC.from_pretrained("cpierse/wav2vec2-large-xlsr-53-esperanto")

In [ ]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

test_dataset = load_dataset("common_voice", "eo", split="test[:2%]", trust_remote_code=True) 
processor = Wav2Vec2Processor.from_pretrained("cpierse/wav2vec2-large-xlsr-53-esperanto") 
model = Wav2Vec2ForCTC.from_pretrained("cpierse/wav2vec2-large-xlsr-53-esperanto") 

resampler = torchaudio.transforms.Resample(48_000, 16_000)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
   speech_array, sampling_rate = torchaudio.load(batch["path"])
   batch["speech"] = resampler(speech_array).squeeze().numpy()
   return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"][:2], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
   logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", test_dataset["sentence"][:2])
